In [2]:
# https://openrouteservice.org/
# https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/get

# username = dconly
# email = dconly@sacog.org

import requests
import geopandas as gpd

api_key_source = r"C:\Users\dconly\GitRepos\GIS-tools\ORS\ors-api-dc.txt"
with open(api_key_source) as f:
    ors_api_key = f.readline()







In [5]:
# Call API to get directions between 2 points by specified mode
# need more elegant way to parse json?

# start location
start_lat = 38.59312635026946
start_lon = -121.4487934112549

# end location
end_lat = 38.592995821193265
end_lon = -121.4568614959717

# options include "cycling-regular", "driving-car", "foot-walking"
travel_mode = "cycling-regular"


# ORS api for directions from start point to end point
api_url = f"https://api.openrouteservice.org/v2/directions/{travel_mode}?api_key={ors_api_key}" \
        f"&start={start_lon},{start_lat}&end={end_lon},{end_lat}"

call = requests.get(api_url)

json_result = call.json()
path_data = json_result['features'][0]['properties']['segments'][0]

path_dist_m = path_data['distance']
path_dur_sec = path_data['duration']

dist_mi = round(path_dist_m * 0.0006213712, 2)
path_dur_mins = round(path_dur_sec / 60, 2)

print(f"{dist_mi} miles in {path_dur_mins} mins")

0.68 miles in 5.29 mins


In [26]:
# Make isochrone around single point
# https://openrouteservice.org/dev/#/api-docs/isochrones

travel_mode = "foot-walking"

orgn_lat = 38.59312635026946
orgn_lon = -121.4487934112549

max_time_mins = 15
max_time_sec = max_time_mins * 60


# IMPORTANT NOTE!!! CAN ENTER ARRAY OF MULTIPLE LAT/LONGS, SO COULD INSERT MULTIPLE POINTS TO GET LINE-BASED ISOCHRONE
body = {"locations":[[orgn_lon, orgn_lat]], "range":[max_time_sec], "range_type":"time"}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': ors_api_key,
    'Content-Type': 'application/json; charset=utf-8'
}

call = requests.post(f'https://api.openrouteservice.org/v2/isochrones/{travel_mode}', json=body, headers=headers)

json = call.json()
txt = call.text
json
# gpd.GeoDataFrame(json['features'], geometry='geometry')

{'type': 'FeatureCollection',
 'bbox': [-121.460692, 38.58504, -121.437824, 38.597321],
 'features': [{'type': 'Feature',
   'properties': {'group_index': 0,
    'value': 900.0,
    'center': [-121.44875188166795, 38.593163819244374]},
   'geometry': {'coordinates': [[[-121.460692, 38.595053],
      [-121.460571, 38.594714],
      [-121.458764, 38.590803],
      [-121.456705, 38.590897],
      [-121.456491, 38.5909],
      [-121.456302, 38.590903],
      [-121.456132, 38.590931],
      [-121.455065, 38.591077],
      [-121.449436, 38.591245],
      [-121.443846, 38.589243],
      [-121.443085, 38.588727],
      [-121.439565, 38.585075],
      [-121.439041, 38.58504],
      [-121.437897, 38.58541],
      [-121.437824, 38.585763],
      [-121.43858, 38.587357],
      [-121.4389, 38.587523],
      [-121.44245, 38.588715],
      [-121.444365, 38.59378],
      [-121.444745, 38.595547],
      [-121.446982, 38.597129],
      [-121.447307, 38.597283],
      [-121.447612, 38.597321],
      [-12

In [35]:
import plotly.express as px

gdf = gpd.read_file(txt)
gdf.head()


fig = px.choropleth(gdf, geojson=gdf.geometry, locations=gdf.index)
fig.update_geos(fitbounds="locations")

fig.show()


In [ ]:
# Make isochrone around multiple points along a line